In [54]:
!pip install -U deep-translator

import re
import nltk
import random
import numpy as np
import pandas as pd
import langid

from deep_translator import GoogleTranslator
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [55]:
tatoeba = pd.read_csv("/kaggle/input/nmt-data/tatoeba.tsv", sep='\t', usecols=[1,3], names=['English', 'Gaeilge'])
dcep = pd.read_csv("/kaggle/input/nmt-data/DCEP.txt", sep='\t', names=['English', 'Gaeilge'])
dgt_tm = pd.read_csv('/kaggle/input/nmt-data/dgt-tm.txt', encoding_errors='ignore', sep='\t', names=['English', 'Gaeilge'], header=0)

pairs = pd.concat([tatoeba, dcep, dgt_tm], ignore_index=True).sample(frac=1)
pairs

,English,Gaeilge
39390,"Subsequently, the President of the Commission ...","Ina dhiaidh sin, cuirfidh Uachtarán an Choimis..."
6656,Members whose election has been notified to Pa...,Déanfaidh na Feisirí a mbeidh a dtoghadh curth...
35223,"Towards the end of the parliamentary term, req...",Agus an Pharlaimint ag druidim le deireadh a t...
22415,"In the event of a disagreement, the matter sha...","I gcás easaontais, tarchuirfear an t-ábhar chu..."
3389,Throughout the negotiations the Commission and...,"I rith na caibidlíochta, cuirfidh an Coimisiún..."
...,...,...
5741,Parliament shall decide by a majority of the v...,Déanfaidh an Pharlaimint cinneadh le tromlach ...
36877,"A committee, an interparliamentary delegation,...","Féadfaidh coiste, toscaireacht idirpharlaimint..."
6118,renewed consultation,renewed consultation
42002,They shall be forwarded to the President after...,"Cuirfear ar aghaidh chuig an Uachtarán iad, ta..."


In [56]:
t = GoogleTranslator(source='en', target='ga')
t.translate("I have a dog and a cat")

'Tá madra agus cat agam'

In [57]:
sample = pairs.sample(10)
sample

,English,Gaeilge
23821,All reports tabled under the legislative proce...,Maidir le gach tuarascáil a chuirtear síos fao...
10331,the legal protection of Parliament's rights an...,cosaint dhlíthiúil chearta agus shainchumais n...
8201,"Europos Parlamento nariai in Lithuanian,","Europos Parlamento nariai sa Liotuáinis,"
28818,Electronic voting,Vótáil leictreonach
42342,"relations with the WTO, including its parliame...",caidreamh leis an Eagraíocht Dhomhanda Trádála...
6855,Duties of the Quaestors,Dualgais na gCaestóirí
17622,(a) If a proposal to grant discharge secures a...,(a) Má fhaigheann togra chun urscaoileadh a dh...
10227,rules on competition and State or public aid;,rialacha ar iomaíocht agus ar chabhair Stáit n...
39283,"Moreover, the Commission may forward any confi...","Ina theannta sin, féadfaidh an Coimisiún aon f..."
13199,In addition to the cases provided for under Ru...,De bhreis ar na cásanna dá bhforáiltear faoi R...


In [58]:
zipped = zip(sample["English"], sample["Gaeilge"])

google = [t.translate(en) for en, ga in zipped]
google

['Ba cheart go gcomhlíonfadh gach tuarascáil a chuirtear síos faoin nós imeachta reachtach Riail 37, 43 agus 45.',
 'cosaint dhlíthiúil chearta agus shainchumais na Parlaiminte, lena n-áirítear a rannpháirtíocht i gcaingne os comhair Chúirt Bhreithiúnais an Aontais Eorpaigh;',
 'Europos Parlamento nariai sa Liotuáinis,',
 'Vótáil leictreonach',
 'caidreamh leis an WTO, lena n-áirítear a ghné pharlaiminteach.',
 'Dualgais na gCaestóirí',
 '(a) Má fhaigheann togra chun urscaoileadh a dheonú tromlach, deonófar urscaoileadh.',
 'rialacha maidir le hiomaíocht agus cúnamh Stáit nó poiblí;',
 'Ina theannta sin, féadfaidh an Coimisiún aon fhaisnéis rúnda ar a thionscnamh féin a chur ar aghaidh chuig an bParlaimint i gcomhréir le forálacha na hIarscríbhinne seo.',
 'I dteannta na gcásanna dá bhforáiltear faoi Riail 106(4), faoi Riail 107(5) agus faoi Riail 166, déanfar an vótáil le glaoch rolla má iarrann grúpa polaitiúil nó 40 Feisire ar a laghad é sin i scríbhinn tráthnóna roimh an vótáil mur

In [59]:
sample["Google"] = google
sample

,English,Gaeilge,Google
23821,All reports tabled under the legislative proce...,Maidir le gach tuarascáil a chuirtear síos fao...,Ba cheart go gcomhlíonfadh gach tuarascáil a c...
10331,the legal protection of Parliament's rights an...,cosaint dhlíthiúil chearta agus shainchumais n...,cosaint dhlíthiúil chearta agus shainchumais n...
8201,"Europos Parlamento nariai in Lithuanian,","Europos Parlamento nariai sa Liotuáinis,","Europos Parlamento nariai sa Liotuáinis,"
28818,Electronic voting,Vótáil leictreonach,Vótáil leictreonach
42342,"relations with the WTO, including its parliame...",caidreamh leis an Eagraíocht Dhomhanda Trádála...,"caidreamh leis an WTO, lena n-áirítear a ghné ..."
6855,Duties of the Quaestors,Dualgais na gCaestóirí,Dualgais na gCaestóirí
17622,(a) If a proposal to grant discharge secures a...,(a) Má fhaigheann togra chun urscaoileadh a dh...,(a) Má fhaigheann togra chun urscaoileadh a dh...
10227,rules on competition and State or public aid;,rialacha ar iomaíocht agus ar chabhair Stáit n...,rialacha maidir le hiomaíocht agus cúnamh Stái...
39283,"Moreover, the Commission may forward any confi...","Ina theannta sin, féadfaidh an Coimisiún aon f...","Ina theannta sin, féadfaidh an Coimisiún aon f..."
13199,In addition to the cases provided for under Ru...,De bhreis ar na cásanna dá bhforáiltear faoi R...,I dteannta na gcásanna dá bhforáiltear faoi Ri...


In [60]:
sample.to_csv("sample.csv")